# Prepare All Data

### Import necessary modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

### Processing component information

### Load training and test datasets

In [2]:
train = pd.read_csv('input/train_set.csv', parse_dates=[2,])
# y_train = train ["cost"].values
# train_df = train_df.drop(['cost'], axis = 1)
# y is converted to log(1+y), RMSLE is then coverted into RMSE for new y
# y_train = np.log1p(y_train)

# generate fake negative id for training data
train['id'] = -1 * np.arange(0, len(train))
train = train.set_index('id')


test = pd.read_csv('input/test_set.csv', parse_dates=[3,], index_col = 'id')
test["cost"] = 0

# merge training data X and test X together 
# so the training data and test data can be preprocessed in the same way 
all_data = pd.concat([train, test])

### Merge tube, specs, and end form data

In [3]:
all_data = all_data.reset_index()

# merge tube data
tube = pd.read_csv('input/tube.csv', na_values = ['NONE', 9999], true_values = 'Y', false_values = 'N')
all_data = pd.merge(all_data, tube, on='tube_assembly_id', how='left')

# merge bom data, bom = bill of material
# bom = pd.read_csv('input/bill_of_materials.csv')
# all_data = pd.merge(all_data, bom, on='tube_assembly_id', how='left')

# specs of the tubes
specs = pd.read_csv('input/specs.csv')
specs[specs.notnull()] = 1
specs = specs.fillna(0)


all_data['spec_num'] = specs[['spec' + str(x) for x in range(1, 10)]].sum(axis = 1)


# merge end forming data
end_form = pd.read_csv('input/tube_end_form.csv')

# all_data.loc[all_data['end_a'] == "NONE", 'end_a_forming'] = -999
# all_data.loc[all_data['end_x'] == "NONE", 'end_x_forming'] = -999

for idx,row in end_form.iterrows():
    if row['forming'] == 'Yes':
        end_forming_value = 1
    if row['forming'] == 'No':
        end_forming_value = 0

    all_data.loc[all_data['end_a'] == row['end_form_id'], 'end_a_forming'] = end_forming_value
    all_data.loc[all_data['end_x'] == row['end_form_id'], 'end_x_forming'] = end_forming_value

bom_comp = pickle.load(open('bom_comp.pkl', 'rb'))
all_data = pd.merge(all_data, bom_comp, on='tube_assembly_id', how='left')
all_data = all_data.set_index('id', drop = True)


float_column = list(all_data.select_dtypes(include=['float64']).columns)
all_data[float_column] = all_data[float_column].fillna(0)

# year and month are treated as feature
all_data['year'] = all_data['quote_date'].dt.year
all_data['month'] = all_data['quote_date'].dt.month
all_data = all_data.drop('quote_date', axis = 1)

# there is soft leak in the assembly id
# assembly id is treated as a feature
all_data['tube_assembly_id'] =  all_data['tube_assembly_id'].str[3: ]
all_data['tube_assembly_id'] = all_data['tube_assembly_id'].astype('int64')

all_data['cross_section'] = all_data['diameter'] ** 2 - (
    all_data['diameter'] - all_data['wall'])  ** 2

# price = fixed price + variable price / quantity
all_data['quantity'] = 1 / all_data['quantity']

In [4]:
#Dump the merged data to use in modeling
pickle.dump(all_data, open('all_data.pkl', 'wb'))

In [5]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60448 entries, 0 to 30235
Columns: 176 entries, tube_assembly_id to cross_section
dtypes: bool(4), float64(158), int64(9), object(5)
memory usage: 80.0+ MB


In [6]:
all_data.head()

,tube_assembly_id,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,cost,material_id,diameter,wall,...,thread_end_form_id,thread_number,total_comp_number,total_weight,min_weight,max_weight,mean_weight,year,month,cross_section
id,,,,,,,,,,,,,,,,,,,,,
0,2,S-0066,0,0,Yes,1.00,21.905933,SP-0019,6.35,0.71,...,0.0,0.0,2.0,0.018,0.018,0.018,0.018,2013,7,8.5129
-1,2,S-0066,0,0,Yes,0.50,12.341214,SP-0019,6.35,0.71,...,0.0,0.0,2.0,0.018,0.018,0.018,0.018,2013,7,8.5129
-2,2,S-0066,0,0,Yes,0.20,6.601826,SP-0019,6.35,0.71,...,0.0,0.0,2.0,0.018,0.018,0.018,0.018,2013,7,8.5129
-3,2,S-0066,0,0,Yes,0.10,4.687770,SP-0019,6.35,0.71,...,0.0,0.0,2.0,0.018,0.018,0.018,0.018,2013,7,8.5129
-4,2,S-0066,0,0,Yes,0.04,3.541561,SP-0019,6.35,0.71,...,0.0,0.0,2.0,0.018,0.018,0.018,0.018,2013,7,8.5129


In [7]:
all_data.tail()

,tube_assembly_id,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,cost,material_id,diameter,wall,...,thread_end_form_id,thread_number,total_comp_number,total_weight,min_weight,max_weight,mean_weight,year,month,cross_section
id,,,,,,,,,,,,,,,,,,,,,
30231,21192,S-0041,1,80,No,0.012500,0.0,SP-0028,25.40,1.65,...,0.0,0.0,1.0,0.118,0.118,0.118,0.118000,2013,8,81.0975
30232,21193,S-0041,868,75,No,0.013333,0.0,SP-0035,15.88,1.65,...,0.0,0.0,3.0,0.274,0.009,0.219,0.091333,2013,9,49.6815
30233,21194,S-0041,883,85,No,0.011765,0.0,SP-0035,15.88,1.65,...,0.0,0.0,3.0,0.274,0.009,0.219,0.091333,2013,9,49.6815
30234,21198,S-0026,28,1,No,1.000000,0.0,SP-0029,101.60,1.65,...,0.0,0.0,0.0,0.000,0.000,0.000,0.000000,2003,9,332.5575
30235,21199,S-0026,28,1,No,1.000000,0.0,SP-0029,101.60,1.65,...,0.0,0.0,0.0,0.000,0.000,0.000,0.000000,2003,9,332.5575


In [8]:
all_data.head()

,tube_assembly_id,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,cost,material_id,diameter,wall,...,thread_end_form_id,thread_number,total_comp_number,total_weight,min_weight,max_weight,mean_weight,year,month,cross_section
id,,,,,,,,,,,,,,,,,,,,,
0,2,S-0066,0,0,Yes,1.00,21.905933,SP-0019,6.35,0.71,...,0.0,0.0,2.0,0.018,0.018,0.018,0.018,2013,7,8.5129
-1,2,S-0066,0,0,Yes,0.50,12.341214,SP-0019,6.35,0.71,...,0.0,0.0,2.0,0.018,0.018,0.018,0.018,2013,7,8.5129
-2,2,S-0066,0,0,Yes,0.20,6.601826,SP-0019,6.35,0.71,...,0.0,0.0,2.0,0.018,0.018,0.018,0.018,2013,7,8.5129
-3,2,S-0066,0,0,Yes,0.10,4.687770,SP-0019,6.35,0.71,...,0.0,0.0,2.0,0.018,0.018,0.018,0.018,2013,7,8.5129
-4,2,S-0066,0,0,Yes,0.04,3.541561,SP-0019,6.35,0.71,...,0.0,0.0,2.0,0.018,0.018,0.018,0.018,2013,7,8.5129
